In [31]:
from datetime import datetime
from pyspark.sql.functions import udf, col, monotonically_increasing_id, lit
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, from_unixtime, dayofweek
from pyspark.sql.functions import to_timestamp, to_date

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date, TimestampType
song_schema = R([
    Fld('artist_id', Str(), True),
    Fld('artist_latitude', Dbl()),
    Fld('artist_location', Str()),
    Fld('artist_longitude', Dbl()),
    Fld('artist_name', Str()),
    Fld('duration', Dbl()),
    Fld('num_songs', Int()),
    Fld('song_id', Str()),
    Fld('title', Str()),
    Fld('year', Int()),
])

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1615746192605_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
song_data = 's3://udacity-dend/song_data/*/*/*/*.json'
df = spark.read.json(song_data, song_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.printSchema()
df.show(5)
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: integer (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)

+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442|          Billy 

# Create Songs Table

In [10]:
song_table = df.select('song_id', 'title', 'artist_id', 'year', 'duration').dropDuplicates()
song_table.printSchema()
song_table.show(5)
song_table.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- duration: double (nullable = true)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOJWWEV12A8C136B97|Last Rose Of Summ...|AR4FBUO1187B9B6072|2005|260.04853|
|SOUJLXS12A3F1ECF59|Catholic Knees (A...|ARG9CID1187B9AB010|2009|168.25424|
|SOBGZZO12A6701CD25|Keep The Circle A...|AR9AM2N1187B9AD2F1|1999|230.05995|
|SOZIUVT12A6D4F67A0|Now I'm Here (Liv...|ARNLO2P1187B9A3ECD|   0|418.48118|
|SOMCMZX12A67AE0221|         The Old Sod|ARBW7IC1187B9908CE|1990|202.63138|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

+-------+------------------+-----------------+------------------+------------------+-------

### Create parquet files

In [11]:
song_table.write.partitionBy('year', 'artist_id').mode("overwrite").parquet('s3://sparkifydb-data/songs')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Artists table

In [12]:
artists_table = df.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude').dropDuplicates()
artists_table.printSchema()
artists_table.show(5)
artists_table.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARLMHFV1187B9A3833|       Smif-n-Wessun|New York, NY [Bro...|       40.65507|       -73.94888|
|ARI66OL1187B9AFA5B|          Wilderness|       Baltimore, MD|       39.29055|        -76.6096|
|ARTNON61187B98D6EE|               Elbow| Manchester, England|           null|            null|
|ARBHW3X1187B999013|La Oreja de Van Gogh|                    |           null|            null|
|ARF3HEO1187B9AF2F4|    Donnie McClurkin|                    |           null|            null|


### Create Parquet files

In [13]:
artists_table.write.mode("overwrite").parquet('s3://sparkifydb-data/artists')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read all log data

In [20]:
log_data = 's3://udacity-dend/log_data/*/*/*.json'
df2 = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df2.printSchema()
df2.show(5)
df2.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            lo

In [26]:
df_nextsong = df2.filter(col("page") == 'NextSong')
df_nextsong.show(5)
df_nextsong.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      

## User Table

In [27]:
users_table = df_nextsong.select('userId', 'firstName', 'lastName', 'gender', 'level').dropDuplicates()
users_table.printSchema()
users_table.show(5)
users_table.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    57|Katherine|     Gay|     F| free|
|    84|  Shakira|    Hunt|     F| free|
|    22|     Sean|  Wilson|     F| free|
|    52| Theodore|   Smith|     M| free|
|    80|    Tegan|  Levine|     F| paid|
+------+---------+--------+------+-----+
only showing top 5 rows

+-------+------------------+---------+--------+------+-----+
|summary|            userId|firstName|lastName|gender|level|
+-------+------------------+---------+--------+------+-----+
|  count|               104|      104|     104|   104|  104|
|   mean| 51.50961538461539|     null|    null|  null| null|
| stddev|29.035296629035354|     null|    null|  null| null|
|    min|                10|   Adelyn|Ar

### create parquet files

In [28]:
users_table.write.mode("overwrite").parquet('s3://sparkifydb-data/users')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Add timestamp column

In [32]:
get_timestamp = udf(lambda x: datetime.fromtimestamp(x / 1000), TimestampType())
df_nextsong = df_nextsong.withColumn("start_time", get_timestamp(df2.ts))
df_nextsong.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|          start_time|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+--------------------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:...|
|The Prodigy|Logged In|     Ryan|     M|            1|  

# Create Time table

In [41]:
time_table = df_nextsong.select('start_time')\
    .withColumn('hour', hour('start_time'))\
    .withColumn('day', dayofmonth('start_time'))\
    .withColumn('week', weekofyear('start_time'))\
    .withColumn('month', month('start_time'))\
    .withColumn('year', year('start_time'))\
    .withColumn('weekday', dayofweek('start_time'))
time_table.show(5)
time_table.printSchema()
time_table.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:41:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:45:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 03:44:...|   3| 15|  46|   11|2018|      5|
|2018-11-15 05:48:...|   5| 15|  46|   11|2018|      5|
+--------------------+----+---+----+-----+----+-------+
only showing top 5 rows

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

+-------+------------------+------------------+------------------+--------------------+------+------------------+
|summary|              hour|               day|              week|               

### Create Parquet files

In [34]:
time_table.write.partitionBy('year', 'month').mode("overwrite").parquet('s3://sparkifydb-data/time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Songplays table

In [35]:
# Read in song data from partitioned parquet files
song_df = spark.read.parquet('s3://sparkifydb-data/songs').select('song_id', 'title', 'artist_id')
song_df.printSchema()
song_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)

+------------------+--------------------+------------------+
|           song_id|               title|         artist_id|
+------------------+--------------------+------------------+
|SODUMDU12AC468A22B|We're Skrewed (Ot...|ARHOSMU1242078130D|
|SOTCIHX12A8C13DDD2|Finally_ as that ...|ARYOIZG1187FB41E30|
|SOPSXLI12A6D4FA418|Practical Cats - ...|ARR79V31187FB5B96E|
|SOUFBFK12A8C13D668|String Quartets O...|ARAILTA11F4C840A06|
|SOROAMT12A8C13C6D0|Me gustan mas los...|ARWUDTF1187B9AA096|
+------------------+--------------------+------------------+
only showing top 5 rows

In [36]:
# Read in artist data from partitioned parquet files
artists_df = spark.read.parquet('s3://sparkifydb-data/artists')
artists_df.printSchema()
artists_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARIBQJ21187FB48B54| Alban Berg Quartett|                    |           null|            null|
|ARV3K201187FB3A680|       Helen Shapiro|Bethnal Green, Lo...|       51.52668|        -0.06257|
|AR3QFYG1187B992B2E|     Michelle Wright|Morpeth, Ontario,...|       42.39336|       -81.84348|
|AR0WBBL1187FB4677D|Cute Is What We A...|         Buffalo, NY|       42.88544|       -78.87846|
|ARZORKK11E2835CDA4|   Vincent Vallières|          Sherbrooke|       45.40326|       -71.88893|


In [38]:
# Create joined dataframe from song_df & artist_df
song_artist = song_df.join(artists_df, ["artist_id"])
song_artist.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+--------------------+---------------+---------------+----------------+
|         artist_id|           song_id|               title|         artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+------------------+--------------------+--------------------+---------------+---------------+----------------+
|ARHOSMU1242078130D|SODUMDU12AC468A22B|We're Skrewed (Ot...|         All Leather|               |           null|            null|
|ARYOIZG1187FB41E30|SOTCIHX12A8C13DDD2|Finally_ as that ...|        Red Sparowes|Los Angeles, CA|       34.05349|      -118.24532|
|ARR79V31187FB5B96E|SOPSXLI12A6D4FA418|Practical Cats - ...|Robert Donat/Phil...|               |           null|            null|
|ARAILTA11F4C840A06|SOUFBFK12A8C13D668|String Quartets O...|Endellion String ...|               |           null|            null|
|ARWUDTF1187B9AA096|SOROAMT12A8C13C6D0|Me gustan mas los...|          Albertucho|  

### Create table

In [42]:
songplays_table = df_nextsong.join(song_artist, (df_nextsong.song == song_artist.title) & (df_nextsong.artist == song_artist.artist_name), 'left')\
                    .join(time_table, ['start_time'])\
                    .select(col("start_time"),
                                col("year"),
                                col("month"),
                                col("userId").alias("user_id"),
                                col("level"),
                                col("song_id"),
                                col("artist_id"),
                                col("sessionId").alias('session_id'),
                                col("location"),
                                col("userAgent").alias("user_agent"))\
                    .withColumn("songplay_id", monotonically_increasing_id())
songplays_table.printSchema()
songplays_table.show(5)
songplays_table.describe().show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- songplay_id: long (nullable = false)

+--------------------+----+-----+-------+-----+-------+---------+----------+--------------------+--------------------+-----------+
|          start_time|year|month|user_id|level|song_id|artist_id|session_id|            location|          user_agent|songplay_id|
+--------------------+----+-----+-------+-----+-------+---------+----------+--------------------+--------------------+-----------+
|2018-11-07 00:17:...|2018|   11|     97| paid|   null|     null|       293|Lansing-East Lans...|"Mozilla/5.0 (X11...|          0|
|2018-11-26 10:52:...|2018|   

### Create Parquet files

In [43]:
songplays_table.write.partitionBy('year', 'month').mode('overwrite').parquet('s3://sparkifydb-data/songplays')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…